In [7]:
# 前三个Cell为基础函数, 之前都有, 直接复制第四个Cell的内容

In [17]:
import mysql.connector as db # Use library "mysql-connector-python"
import re
import numpy  as np
import pandas as pd
import math

In [18]:
class DBConnectionError(Exception):
    def __init__(self,n): 
        self.n=n

class SQLQueryError(Exception):
    def __init__(self,n): 
        self.n=n

In [22]:
# The basic function for database connection
# Do not use it directly

def mysql_connection(ip_port, user, passwd, schema):
# ip_port should be like this: ip:port
    ip_port_split = re.search("(.*):(\d*)", ip_port)
    ip   = ip_port_split[1]
    port = ip_port_split[2]
    
    
    try:
        connection = db.connect(host=ip, port=port,
                                user=user, passwd=passwd, 
                                db=schema) # specify the schema

        cursor = connection.cursor()
    except mysql.connector.Error as msg: 
        raise DBConnectionError(msg)
    return connection,cursor

In [28]:
def ranking_query_data(rank_type):
# input format
# type: world, league, all, fifa, uefa

    conn, cur = mysql_connection("192.168.56.102:6123", "root", "mysql", "football")
    
    if(len(rank_type)!=4):
    # use word length to judge whether is going to use our table or uefa / fifa table
        format_str="""
            SELECT NAME, points
            FROM {rank_type}_country
            ORDER BY points DESC;"""
        query = format_str.format(rank_type=rank_type)
    
    elif(rank_type=="fifa"):
        query="""
            SELECT TEAM, TOTAL_POINTS
            FROM std_fifa_ranking;"""
        
    else:
        query="""
            SELECT Country, Pts
            FROM std_uefa_ranking;"""
        
    print(query) # for debug
    
    try:
        cur.execute(query)
    except mysql.connector.ProgrammingError as msg:
        raise SQLQueryError(msg)
    
    try:
        results = cur.fetchall()
    except InterfaceError as msg:
        if(msg=="No result set to fetch from."): return None
    
    df_results = pd.DataFrame(results, columns=["Country","Points"])
    df_results.index.name = 'ranking' 
    return df_results

# display(ranking_query_data("world"))
# display(ranking_query_data("league"))
# display(ranking_query_data("all"))

# display(ranking_query_data("fifa"))
# display(ranking_query_data("uefa"))


            SELECT NAME, points
            FROM world_country
            ORDER BY points DESC;


,Country,Points
ranking,,
0,Brazil,1791.42
1,France,1784.10
2,Belgium,1758.81
3,Spain,1726.03
4,Germany,1682.22
...,...,...
307,Sri Lanka,727.87
308,East Timor,718.70
309,Liechtenstein,686.58



            SELECT NAME, points
            FROM league_country
            ORDER BY points DESC;


,Country,Points
ranking,,
0,england,1726.07
1,spain,1639.48
2,netherlands,1525.06
3,portugal,1482.28
4,germany,1471.53
...,...,...
206,estonia,807.08
207,northern-ireland,802.29
208,faroe-islands,748.19



            SELECT NAME, points
            FROM all_country
            ORDER BY points DESC;


,Country,Points
ranking,,
0,England,1836.67
1,Spain,1750.07
2,Netherlands,1636.08
3,Brazil,1628.14
4,Portugal,1593.16
...,...,...
354,Laos,713.79
355,Bhutan,690.01
356,Andorra,541.71



            SELECT TEAM, TOTAL_POINTS
            FROM std_fifa_ranking;


,Country,Points
ranking,,
0,Belgium,1783.38
1,France,1757.3
2,Brazil,1742.65
3,England,1686.78
4,Portugal,1666.12
...,...,...
205,Turks and Caicos Islands,843.65
206,US Virgin Islands,829.13
207,British Virgin Islands,826.27



            SELECT Country, Pts
            FROM std_uefa_ranking;


,Country,Points
ranking,,
0,England,100.140
1,Spain,97.712
2,Italy,75.438
3,Germany,73.570
4,France,56.081
5,Portugal,48.549
6,Netherlands,39.200
7,Russia,38.382
8,Belgium,36.500
